# <ins>C</ins>alibrated <ins>E</ins>xplanations for Regression (ce)
## Demonstrated on the Californa Housing data set

Author: Tuwe Löfström (tuwe.lofstrom@ju.se)  
Copyright 2023 Tuwe Löfström  
License: BSD 3 clause
Sources:
1. [California Housing data set [kaggle]](https://www.kaggle.com/datasets/camnugent/california-housing-prices)
2. [Calibrated Explanations for Regression [Arxiv]](https://arxiv.org/pdf/....pdf)

### 1. Import packages, data and train an underlying model

#### 1.1 Import packages

In the examples below, we will be using `NumPy`, `pandas`, `sklearn`, and `crepes`. From `crepes`, we import `ConformalPredictiveSystem` and some helper functions from `crepes.extras`. `CalibratedExplainer` is imported from `ce`.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from calibrated_explanations import CalibratedExplainer

from crepes import ConformalPredictiveSystem
from crepes.extras import DifficultyEstimator 

#### 1.2 Import data and train a model
Let us import the Califronia Housing data set (see sources at the top).

In [ ]:

dataSet = 'housing.csv'
delimiter = ';'
categorical_labels = {8: {0: 'INLAND', 1: 'NEAR BAY', 2: '<1H OCEAN', 3: 'NEAR OCEAN', 4: 'ISLAND'}}

fileName = '../data/reg/' + dataSet
df = pd.read_csv(fileName, delimiter=delimiter, dtype=np.float64)
target = 'median_house_value'
df.dropna(inplace=True)
X, y = df.drop(target,axis=1), df[target] 
feature_names = df.drop(target,axis=1).columns
df.head()

Let us split the data set into a training and a test set, and further split the training set into a proper training set and a calibration set. Let us fit a random forest to the proper training set. We also set a random seed to be able to rerun the notebook and get the same results.

In [ ]:
num_to_test = 1

trainCalX, testX, trainCalY, testY = train_test_split(X.values, y.values, test_size=num_to_test, random_state=42)
trainX, calX, trainY, calY = train_test_split(trainCalX, trainCalY, test_size=0.33, random_state=42)

model = RandomForestRegressor(n_estimators=100, oob_score=True)

model.fit(trainX,trainY)  

Before extracting an explanation, lets see what the output from a Conformal Predictive System (cps) is for the test instances. Three use cases are explored:
1. using percentiles to get the lower and upper bounds of an interval.
2. using the 50th percentile(s) to get the median prediction which can be used as a calibrated prediction from the underlying model.
3. getting the probability of the prediction being below a certain threshold. 250 000 is used as threshold, as it is close to the midpoint in the possible range of prices.

In [ ]:
calY_pred = model.predict(calX)
testY_pred = model.predict(testX)

cps = ConformalPredictiveSystem()
cps.fit(residuals=calY-calY_pred)

interval = cps.predict(y_hat=testY_pred, lower_percentiles=[5,50], higher_percentiles=[95,50])

p_values = cps.predict(y_hat=testY_pred, y=250000)

print('5th percentile, prediction, 50th percentile, 95th percentile, true value, p-value (cpsd<250000)')
print(*zip(np.round(interval[:,0], decimals=1), 
           np.round(testY_pred, decimals=1), 
           np.round((interval[:,1] + interval[:,3])/2, decimals=1), 
           np.round(interval[:,2], decimals=1), 
           np.round(testY,decimals=1), 
           np.round(p_values*100, decimals=1)),sep='\n')  

In [ ]:
ce = CalibratedExplainer(model, 
                        calX, 
                        calY, 
                        feature_names=feature_names, 
                        categorical_labels=categorical_labels,
                        mode='regression')  
display(ce)

In [ ]:
tsi = (10,90) # two-sided interval with 80% confidence

In [ ]:
discretizer = 'binary'
ce.set_discretizer(discretizer)
exp = ce(testX, 180000, low_high_percentiles=tsi)

exp.add_conjunctive_factual_rules().plot_regular()

In [ ]:
exp.remove_conjunctive_rules().plot_regular()

In [ ]:
discretizer = 'decile'
ce.set_discretizer(discretizer)
exp = ce(testX, 180000, low_high_percentiles=tsi)

exp.add_conjunctive_counterfactual_rules().plot_counterfactuals()

In [ ]:
exp.remove_conjunctive_rules().plot_counterfactuals()

#### Normalized explanations using knn

There are three ways to set the difficulty estimator (sigma) for the CalibratedExplainer when using k-nearest neighbor.
* alternative 1: by the (Euclidean) distances to the nearest neighbors
* alternative 2: by the standard deviation of the targets of the nearest neighbors
* alternative 3: by the absolute errors of the k nearest neighbors

See the documentation of crepes for further details, see [here](https://crepes.readthedocs.io/en/latest/).

In [ ]:
# alternative 1: by the (Euclidean) distances to the nearest neighbors
ce.set_difficultyEstimator(DifficultyEstimator().fit(X=trainX, scaler=True))

# alternative 2: by the standard deviation of the targets of the nearest neighbors
ce.set_difficultyEstimator(DifficultyEstimator().fit(X=trainX, y=trainY, scaler=True))

# alternative 3: by the absolute errors of the k nearest neighbors
oob_predictions = model.oob_prediction_ # requires the model to have been trained with oob_score=True, available for RandomForestRegressor
residuals_oob = trainY - oob_predictions
ce.set_difficultyEstimator(DifficultyEstimator().fit(X=trainX, residuals=residuals_oob, scaler=True))

In [ ]:
discretizer = 'binary'
ce.set_discretizer(discretizer)
exp = ce(testX, 180000, low_high_percentiles=tsi)

exp.add_conjunctive_factual_rules(num_to_include=5).plot_regular(num_to_show=15, save_ext=[])

In [ ]:
exp.remove_conjunctive_rules().plot_regular(num_to_show=10, save_ext=[])

#### Normalized counterfactuals using knn

In [ ]:
discretizer = 'decile'
ce.set_discretizer(discretizer)
exp = ce(testX, 180000, low_high_percentiles=tsi)

exp.add_conjunctive_counterfactual_rules().plot_counterfactuals()

In [ ]:
exp.remove_conjunctive_rules().plot_counterfactuals()

#### Normalized explanations using variance

In [ ]:
ce.set_difficultyEstimator(DifficultyEstimator().fit(X=trainX, learner=model, scaler=True))

In [ ]:
discretizer = 'binary'
ce.set_discretizer(discretizer)
exp = ce(testX, 180000, low_high_percentiles=tsi)

exp.add_conjunctive_factual_rules().plot_regular()

In [ ]:
exp.remove_conjunctive_rules().plot_regular(num_to_show=10, save_ext=[])

#### Normalized counterfactuals using variance

In [ ]:
discretizer = 'decile'
ce.set_discretizer(discretizer)
exp = ce(testX, 180000, low_high_percentiles=tsi)

exp.add_conjunctive_counterfactual_rules().plot_counterfactuals()

In [ ]:
exp.remove_conjunctive_rules().plot_counterfactuals()